## Create Sampled Dataset of A2D2


In [22]:
import argparse
import os
import yaml
import numpy as np
import numpy.linalg as la
import open3d as o3d
import pandas as pd
from matplotlib import pyplot as plt
from tqdm.auto import tqdm, trange
from tqdm.contrib.concurrent import process_map, thread_map
from matplotlib import cm
from functools import partial
import concurrent.futures
import random

from sphere import Sphere
from dh_grid import DHGrid
from laserscan import SemLaserScan

import os
import time
import matplotlib.pyplot as plt
import numpy as np
import json
import pprint
from os.path import join
import glob
import cv2
import open3d as o3d
from plyfile import PlyData, PlyElement


%matplotlib inline
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [13]:
dataroot = '/tmp/rslidar/'

cloud_ds = f'{dataroot}/clouds'
export_ds = f'{dataroot}/processed'
elements = os.listdir(cloud_ds)

print(f'Found {len(elements)} elements in the folder.')

Found 120 elements in the folder.


In [31]:
def retrieve_cloud(lidar_file):    
    plydata = PlyData.read(lidar_file)
    vertex = plydata['vertex']
    x = vertex['x']
    y = vertex['y']
    z = vertex['z']
    i = vertex['intensity']
    c = np.zeros_like(i)
    return np.column_stack((x,y,z,i,c))


all_sem_clouds = []
for ele in tqdm(elements):
    cloud_path = f'{cloud_ds}/{ele}'
    # pcd = o3d.io.read_point_cloud(cloud_path)    
    # all_sem_clouds.append(np.asarray(pcd.points))
    
    pcd = retrieve_cloud(cloud_path)
    all_sem_clouds.append(pcd)
print(f'Found {len(all_sem_clouds)} clouds.')

  0%|          | 0/120 [00:00<?, ?it/s]

Found 120 clouds.


In [24]:
# np.reshape(all_sem_clouds[0], (-1, 4))
# all_sem_clouds[0].shape
# all_sem_clouds[0] = np.reshape(all_sem_clouds[0], (-1, 4))

(33832, 4)

In [32]:
def progresser(sample_idx, grid, auto_position=True, write_safe=False, blocking=True, progress=False):
    sample = all_sem_clouds[sample_idx]
    sample_sphere = Sphere(sample)
    features = sample_sphere.sampleUsingGrid(grid)
    return features

bw = 50
grid, _ = DHGrid.CreateGrid(bw)
print(f"Loading complete. Computing features...")
# parallel
sem_idx = np.arange(0, len(all_sem_clouds))
sample_func = partial(progresser, grid=grid)
sem_features = process_map(sample_func, sem_idx, max_workers=16)            

filename = f"{export_ds}/clouds.npy"
np.save(filename, sem_features)
print(f"Wrote features to {filename}.")

filename = f'archive'
np.save(f'{export_ds}/{filename}.npy', all_sem_clouds)
print(f'Saved clouds to {export_ds}/{filename}.npy')

Loading complete. Computing features...


  0%|          | 0/120 [00:00<?, ?it/s]

Wrote features to /tmp/rslidar//processed/clouds.npy.
Saved clouds to /tmp/rslidar//processed/archive.npy


/usr/lib/python3/dist-packages/numpy/lib/npyio.py:528: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  arr = np.asanyarray(arr)


## Visualize extracted pointclouds

In [25]:
def mapIntensityToRGB(i):
    return cm.jet(plt.Normalize(min(i), max(i))(i))

def visualizeRawPointcloud(pcl, val):
    pcd = o3d.geometry.PointCloud()
    pcd.points = o3d.utility.Vector3dVector(pcl[:, 0:3])
    colors = mapIntensityToRGB(val)
#     colors = scan.sem_color_lut[pcl[:,4].astype(np.int)]
    pcd.colors = o3d.utility.Vector3dVector(colors[:,0:3])
    o3d.visualization.draw_geometries([pcd], width=640,  height=480)    
    
def createGrid_old(bw):
    n_grid = 2 * bw
    k = 0;
    points = np.empty([n_grid * n_grid, 2])
    for i in range(n_grid):
        for j in range(n_grid):
            points[k, 0] = (np.pi*(2*i+1))/(4*bw)
            points[k, 1] = (2*np.pi*j)/(2*bw);
            k = k + 1;
    return points
    
def convertGridToEuclidean_old(grid):
    cart_grid = np.zeros([ grid.shape[0], 3])
    cart_grid[:,0] = np.multiply(np.sin(grid[:,0]), np.cos(grid[:,1]))
    cart_grid[:,1] = np.multiply(np.sin(grid[:,0]), np.sin(grid[:,1]))
    cart_grid[:,2] = np.cos(grid[:,0])
    return cart_grid

def create_sampling_sphere(bw):
    grid = createGrid_old(bw)
    xyz_grid = convertGridToEuclidean_old(grid)
    intensities = np.zeros((xyz_grid.shape[0],1))
    sampling_grid = np.hstack((xyz_grid, np.ones((xyz_grid.shape[0], 1), dtype=xyz_grid.dtype)))
    return sampling_grid.T

In [26]:
pc = all_sem_clouds[0]
visualizeRawPointcloud(pc, pc[:, 3])

In [7]:
classes = np.unique(pc[:, 4])
print(f'we have these classes: {classes}')

we have these classes: [ 0.  1.  2.  8.  9. 20. 27. 29. 34. 35. 39. 43. 45. 48. 50. 51. 52.]


In [34]:
cur_sem_cloud = sem_features[110]

cur_sem_cloud = np.reshape(cur_sem_cloud, (3, -1)).T
pc = create_sampling_sphere(bw)
points_xyz = pc.T[:,0:3]
points_xyzl = np.column_stack((points_xyz, cur_sem_cloud[:,1]))

visualizeRawPointcloud(points_xyzl, points_xyzl[:, 3])